In [ ]:
## Load Dataframe
import pandas as pd
pd.options.mode.chained_assignment = None
import os

df_arr = []
root = ".dataDir"
for path, subdirs, files in os.walk(root):
    for name in files:
        df = pd.read_csv(os.path.join(path, name))
        df_arr.append(df)

df_cons = pd.concat(df_arr, ignore_index=True)
display(df_cons)

In [ ]:
## create tenant specific timeseriesplots
import plotly.express as px
customer_name='exl'
df = df_cons.groupby(['ts', 'tenant']).agg({'dataScannedInGB': 'sum', 'processingTimeinHrs':'sum', 'numberOfFilesScanned':'sum', 'IdleTimeInHrs':'sum', 'uniqPodCount':'mean'})
df = df.add_suffix('_Count').reset_index()
df_tenn = df[df['tenant'].str.contains(customer_name, regex=False)]
df_tenn['ts'] = pd.to_datetime(df_tenn['ts'],unit='ms')
df_tenn['numberOfFilesScanned_Count'] = (df_tenn['numberOfFilesScanned_Count']/1000).astype(float)
df_tenn['uniqPodCount_Count'] = (df_tenn['uniqPodCount_Count']*100).astype(float)
df_tenn.dtypes
fig = px.area(df_tenn, x='ts', y=df_tenn.columns[2:7], log_y=False, title=customer_name)
fig.show()
df_tenn.corr(numeric_only=True)
df_tenn.head()

In [ ]:
## Run Aggregations
df_aggr = df_cons.groupby(['ds','tenant','pod']).agg({'dataScannedInGB': 'sum', 'processingTimeinHrs':'sum'})
df_aggr['AvgDataInGBperdayByDw']=df_aggr.dataScannedInGB*24/df_aggr.processingTimeinHrs
df_aggr = df_aggr.add_suffix('_Count').reset_index()
df_aggr.describe(percentiles=[0.25, 0.75])

In [ ]:
## Create Plots
df_aggr = df_cons.groupby(['ds','tenant','pod']).agg({'dataScannedInGB': 'sum', 'processingTimeinHrs':'sum'})
df_aggr['AvgDataInGBperdayByDw']=df_aggr.dataScannedInGB*24/df_aggr.processingTimeinHrs
df_aggr = df_aggr.add_suffix('_Count').reset_index()
fig = px.box(df_aggr, x='ds', y='AvgDataInGBperdayByDw_Count', color='ds', points=False, log_y=True)
fig.update_layout(
    font=dict(family="Didact Gothic"),
    yaxis_title="<b>GB/day/dw<b>",
    xaxis_title="<b>fileFormat<b>",
    title_x=0.5,
    title_font=dict(size=24,color='black'),
    plot_bgcolor='rgba(0, 0, 0, 0)', #'white',  
    paper_bgcolor='rgba(0, 0, 0, 0)', #'white',
    width=1200,  
    height=900
)
fig.update_traces(quartilemethod="exclusive")
fig.update_yaxes(nticks=10, minor=dict(showgrid=True, gridwidth=2, gridcolor='Black'))
fig.show()